In [1]:
!pip install megnet -qq

  ERROR: Command errored out with exit status 1:
   command: 'C:\ProgramData\Anaconda3\python.exe' 'C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\pep517\_in_process.py' build_wheel 'C:\Users\MUHAMM~1\AppData\Local\Temp\tmpytib8dnm'
       cwd: C:\Users\Muhammad Ahmed\AppData\Local\Temp\pip-install-a9sak9d5\spglib_52e3d420be284c71bad5433410a066d9
  Complete output (12 lines):
  setuptools is used.
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\spglib
  copying spglib\spglib.py -> build\lib.win-amd64-3.8\spglib
  copying spglib\__init__.py -> build\lib.win-amd64-3.8\spglib
  running build_ext
  building 'spglib._spglib' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  ----------------------------------------
  ERROR: Failed building wheel for spglib
ERROR: Could n

In [2]:
!pip install pymatgen -qq

  ERROR: Command errored out with exit status 1:
   command: 'C:\ProgramData\Anaconda3\python.exe' 'C:\ProgramData\Anaconda3\lib\site-packages\pip\_vendor\pep517\_in_process.py' build_wheel 'C:\Users\MUHAMM~1\AppData\Local\Temp\tmpg5s8__qk'
       cwd: C:\Users\Muhammad Ahmed\AppData\Local\Temp\pip-install-i_zv9nz_\spglib_6b8905209a9a4989aaa13b736706ffb8
  Complete output (12 lines):
  setuptools is used.
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\spglib
  copying spglib\spglib.py -> build\lib.win-amd64-3.8\spglib
  copying spglib\__init__.py -> build\lib.win-amd64-3.8\spglib
  running build_ext
  building 'spglib._spglib' extension
  error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
  ----------------------------------------
  ERROR: Failed building wheel for spglib
ERROR: Could n

In [3]:
import yaml
import json

import pandas as pd
import numpy as np
from pathlib import Path
import tensorflow as tf

from pathlib import Path
from pymatgen.core import Structure
from sklearn.model_selection import train_test_split
from megnet.models import MEGNetModel
from megnet.data.crystal import CrystalGraph

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
class config:
    datapath = 'idao_2022_data/dichalcogenides_public/dichalcogenides_public'
    test_datapath = 'idao_2022_data/dichalcogenides_private/dichalcogenides_private'
    checkpoint_path = 'idao_2022_data/val_mae_00014_0.028897.hdf5'
    epochs = 800
    batch_size = 128
    lr = 2e-4 
    cutoff = 4

In [ ]:
def read_pymatgen_dict(file):
    with open(file, "r") as f:
        d = json.load(f)
    return Structure.from_dict(d)

en = None
def energy_within_threshold(prediction, target):
    global en
    # compute absolute error on energy per system.
    # then count the no. of systems where max energy error is < 0.02.
    e_thresh = 0.02
    error_energy = tf.math.abs(target - prediction)
    en = error_energy
    success = tf.math.count_nonzero(error_energy < e_thresh)
    total = tf.size(target)
    return success / tf.cast(total, tf.int64)

def prepare_dataset(dataset_path):
    dataset_path = Path(dataset_path)
    targets = pd.read_csv(dataset_path / "targets.csv", index_col=0)
    struct = {
        item.name.strip(".json"): read_pymatgen_dict(item)
        for item in (dataset_path / "structures").iterdir()
    }

    data = pd.DataFrame(columns=["structures"], index=struct.keys())
    data = data.assign(structures=struct.values(), targets=targets)

    return train_test_split(data, test_size=0.25, random_state=666)

 
def prepare_model(cutoff, lr):
    nfeat_bond = 10
    r_cutoff = cutoff
    gaussian_centers = np.linspace(0, r_cutoff + 1, nfeat_bond)
    gaussian_width = 0.8
    model = MEGNetModel(
        graph_converter=CrystalGraph(cutoff=r_cutoff),
        centers=gaussian_centers,
        width=gaussian_width,
        loss=["MSE"],
        npass=2,
        lr=lr,
        metrics=energy_within_threshold
    )
#     ['Eform_MP_2019', 'Eform_MP_2018', 'Efermi_MP_2019', 'Bandgap_classifier_MP_2018', 'Bandgap_MP_2018', 'logK_MP_2018', 'logG_MP_2018', 'logK_MP_2019', 'logG_MP_2019']
#     best = Efermi_MP_2019 some others: 'Eform_MP_2019', 'Eform_MP_2018'
    model = model.from_file("mvl_models/mvl_models/mp-2019.4.1/efermi.hdf5") 
    model.loss = ['MAE']
    model.metrics = energy_within_threshold
    return model

In [ ]:
dataset_path = Path(config.datapath)

In [ ]:
struct = {item.name.strip('.json'): read_pymatgen_dict(item) for item in (dataset_path/'structures').iterdir()}

In [ ]:
import tqdm

In [ ]:
models_path = [
    'Efermi_MP_2019_Weights/val_mae_00011_0.009874_fold0.hdf5',
    'Efermi_MP_2019_Weights/val_mae_00025_0.005614_fold1.hdf5',
    'Efermi_MP_2019_Weights/val_mae_00013_0.007578_fold2.hdf5',
    'Efermi_MP_2019_Weights/val_mae_00016_0.007441_fold3.hdf5',
    'Efermi_MP_2019_Weights/val_mae_00007_0.007703_fold4.hdf5',
    
    'Efermi_MP_2019_Weights/val_mae_00007_0.010768_fold0.hdf5',
    'Efermi_MP_2019_Weights/val_mae_00023_0.005869_fold1.hdf5',
    'Efermi_MP_2019_Weights/val_mae_00014_0.008344_fold2.hdf5',
    'Efermi_MP_2019_Weights/val_mae_00019_0.007814_fold3.hdf5',
    'Efermi_MP_2019_Weights/val_mae_00007_0.008662_fold4.hdf5'
]

In [ ]:
public_test = pd.DataFrame(columns=['id', 'structures'], index=struct.keys())
public_test = public_test.assign(structures=struct.values())
predictions =[]
for path in tqdm.tqdm(models_path):
    model = prepare_model(
        float(config.cutoff), float(config.lr)
    )
    model.load_weights(path)
    predictions.append(model.predict_structures(public_test.structures))

In [ ]:
predictions

In [ ]:
pseudo = pd.DataFrame()
pseudo['id'] = public_test['id'].index.values
pseudo['Pred0'] = pd.DataFrame(predictions[0])
pseudo['Pred1'] = pd.DataFrame(predictions[1])
pseudo['Pred2'] = pd.DataFrame(predictions[2])
pseudo['Pred3'] = pd.DataFrame(predictions[3])
pseudo['Pred4'] = pd.DataFrame(predictions[4])
pseudo['Pred5'] = pd.DataFrame(predictions[5])
pseudo['Pred6'] = pd.DataFrame(predictions[6])
pseudo['Pred7'] = pd.DataFrame(predictions[7])
pseudo['Pred8'] = pd.DataFrame(predictions[8])
pseudo['Pred9'] = pd.DataFrame(predictions[9])

In [ ]:
truth = pd.read_csv('idao_2022_data/dichalcogenides_public/dichalcogenides_public/targets.csv')
truth = truth.rename({'_id':'id'},axis=1)

In [ ]:
pseudo = pd.merge(pseudo, truth, on ='id').drop('id',axis=1)

In [ ]:
# pseudo.to_csv('pseudo.csv', index = False)

In [ ]:
X = pseudo[pseudo.columns[:-1]].copy()
y = pseudo['band_gap'].copy()

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from catboost import CatBoostClassifier, CatBoostRegressor
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMRegressor, LGBMClassifier

from sklearn.linear_model import LogisticRegressionCV, LogisticRegression, LinearRegression, LassoLarsCV, BayesianRidge, SGDClassifier, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier, AdaBoostRegressor, ExtraTreesClassifier, ExtraTreesRegressor, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit

import warnings
warnings.filterwarnings("ignore")

class AutoML:
    def __init__(self):
        self.clf_models = [LogisticRegression(max_iter=10000), 
                           DecisionTreeClassifier(), 
                           KNeighborsClassifier(), SGDClassifier(),  RandomForestClassifier(n_jobs=-1),  
                           AdaBoostClassifier(),  
                           ExtraTreesClassifier(n_jobs=-1),  XGBClassifier(n_jobs=-1),  LGBMClassifier(n_jobs=-1),  
                           CatBoostClassifier(verbose=0),  
                           GradientBoostingClassifier(),  GaussianNB(),  MLPClassifier(max_iter = 1000)  ]
        self.reg_models = [LassoLarsCV(),  LinearRegression(),  DecisionTreeRegressor(),  KNeighborsRegressor(),  
                          SGDRegressor(),  RandomForestRegressor(n_jobs=-1),  AdaBoostRegressor(),  ExtraTreesRegressor(n_jobs=-1),  
                          XGBRegressor(n_jobs=-1),  LGBMRegressor(n_jobs=-1),  CatBoostRegressor(verbose=0),  
                          GradientBoostingRegressor(),  
                          BayesianRidge(),  MLPRegressor(max_iter = 1000) ]
    def plot_table(self, table):
        fig, axs = plt.subplots(1,1, figsize=(15,2))
        collabel=tuple(table.columns)
        axs.axis('tight')
        axs.axis('off')
        the_table = axs.table(cellText=table.values,colLabels=collabel,loc='upper center')
        return fig
    def plot_learning_curve(self,estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5), name = None):
        """
        Generate 3 plots: the test and training learning curve, the training
        samples vs fit times curve, the fit times vs score curve.
        Parameters
        ----------
        estimator : object type that implements the "fit" and "predict" methods
            An object of that type which is cloned for each validation.
        title : string
            Title for the chart.
        X : array-like, shape (n_samples, n_features)
            Training vector, where n_samples is the number of samples and
            n_features is the number of features.
        y : array-like, shape (n_samples) or (n_samples, n_features), optional
            Target relative to X for classification or regression;
            None for unsupervised learning.
        axes : array of 3 axes, optional (default=None)
            Axes to use for plotting the curves.
        ylim : tuple, shape (ymin, ymax), optional
            Defines minimum and maximum yvalues plotted.
        cv : int, cross-validation generator or an iterable, optional
            Determines the cross-validation splitting strategy.
            Possible inputs for cv are:
              - None, to use the default 5-fold cross-validation,
              - integer, to specify the number of folds.
              - :term:`CV splitter`,
              - An iterable yielding (train, test) splits as arrays of indices.
            For integer/None inputs, if ``y`` is binary or multiclass,
            :class:`StratifiedKFold` used. If the estimator is not a classifier
            or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.
            Refer :ref:`User Guide <cross_validation>` for the various
            cross-validators that can be used here.
        n_jobs : int or None, optional (default=None)
            Number of jobs to run in parallel.
            ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
            ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
            for more details.
        train_sizes : array-like, shape (n_ticks,), dtype float or int
            Relative or absolute numbers of training examples that will be used to
            generate the learning curve. If the dtype is float, it is regarded as a
            fraction of the maximum size of the training set (that is determined
            by the selected validation method), i.e. it has to be within (0, 1].
            Otherwise it is interpreted as absolute sizes of the training sets.
            Note that for classification the number of samples usually have to
            be big enough to contain at least one sample from each class.
            (default: np.linspace(0.1, 1.0, 5))
        """
        print(name)
        if axes is None:
            fig, axes = plt.subplots(1, 3, figsize=(20, 5))

        axes[0].set_title(title)
        if ylim is not None:
            axes[0].set_ylim(*ylim)
        axes[0].set_xlabel("Training examples")
        axes[0].set_ylabel("Score")

        train_sizes, train_scores, test_scores, fit_times, _ = \
            learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                           train_sizes=train_sizes,
                           return_times=True)
        train_scores_mean = np.mean(train_scores, axis=1)
        train_scores_std = np.std(train_scores, axis=1)
        test_scores_mean = np.mean(test_scores, axis=1)
        test_scores_std = np.std(test_scores, axis=1)
        fit_times_mean = np.mean(fit_times, axis=1)
        fit_times_std = np.std(fit_times, axis=1)

        # Plot learning curve
        axes[0].grid()
        axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                             train_scores_mean + train_scores_std, alpha=0.1,
                             color="r")
        axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                             test_scores_mean + test_scores_std, alpha=0.1,
                             color="g")
        axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                     label="Training score")
        axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                     label="Cross-validation score")
        axes[0].legend(loc="best")

        # Plot n_samples vs fit_times
        axes[1].grid()
        axes[1].plot(train_sizes, fit_times_mean, 'o-')
        axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                             fit_times_mean + fit_times_std, alpha=0.1)
        axes[1].set_xlabel("Training examples")
        axes[1].set_ylabel("fit_times")
        axes[1].set_title("Scalability of the model")

        # Plot fit_time vs score
        axes[2].grid()
        axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
        axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                             test_scores_mean + test_scores_std, alpha=0.1)
        axes[2].set_xlabel("fit_times")
        axes[2].set_ylabel("Score")
        axes[2].set_title("Performance of the model")

        return np.mean(test_scores_mean)
    
    def cross_val(self, X, y, model, metric, folds):
        scores = []
        for tr_in, val_in in KFold(n_splits = folds).split(X, y):
            model_fold = model
            X_train, y_train, X_val, y_val = X.iloc[tr_in,:], y[tr_in], X.iloc[val_in,:], y[val_in]
            model_fold.fit(X_train, y_train)
            y_hat = model.predict(X_val)
            scores.append(metric(y_val,y_hat))
        return np.mean(scores)
    
    def GMLClassifier(self, X, y, metric = accuracy_score, folds = 5):
        '''
        X: Independent variable(s)
        y: Dependent variable
        metric: metric of evaluation 
        folds: Number of validation folds, default: 5
        '''
        result = pd.DataFrame(columns = ['Models','Scores'])
        for i,model in enumerate(self.clf_models):
            name = str(model.__class__.__name__)
            scores = self.cross_val(X, y, model, metric, folds)
            
            print('{} got score of {} in {} folds'.format(name,scores,folds))
            result.at[i, 'Models'] = name
            result.at[i, 'Scores'] = scores
        result = result.sort_values('Scores',ascending=False).reset_index(drop=True)
        self.plot_table(result)
    
    def GMLRegressor(self, X, y, metric = mean_absolute_error,folds = 5):
        '''
        X: Independent variable(s)
        y: Dependent variable
        metric: metric of evaluation 
        folds: Number of validation folds, default: 5
        '''
        result = pd.DataFrame(columns = ['Models','Scores'])
        for i,model in enumerate(self.reg_models):
            name = str(model.__class__.__name__)
            scores = self.cross_val(X, y, model, metric, folds)
            
            print('{} got score of {} in {} folds'.format(model.__class__.__name__,scores,folds))
            result.at[i, 'Models'] = name
            result.at[i, 'Scores'] = scores
        result = result.sort_values('Scores',ascending=False).reset_index(drop=True)
        self.plot_table(result)

In [ ]:
gml_ml = AutoML()

gml_ml.GMLRegressor(X, y, metric = energy_within_threshold, folds = 10)

In [ ]:
def rounding(num):
    round_num, round_num2 = 0, 0
    uniques = np.sort(np.unique(y))
    for i,n in enumerate(uniques):
        if n > num:
            break
        round_num = n
        try:
            round_num2 = uniques[i+1]
        except:
            round_num2 = round_num
    if np.abs(num - round_num) < np.abs(num - round_num2):
        return round_num
    return round_num2

In [ ]:
def cross_val(X, y, model, metric, folds):
    scores = []
    i = 0
    for tr_in, val_in in KFold(n_splits = folds).split(X, y):
        X_train, y_train, X_val, y_val = X.iloc[tr_in,:], y[tr_in], X.iloc[val_in,:], y[val_in]
        model.fit(X_train, y_train)

        y_hat = model.predict(X_val)

        sc = metric(y_val,y_hat)
        print(sc)
        scores.append(sc)
#         if i == 3:
#             break
        i += 1
    return np.mean(scores)

In [ ]:
cross_val(X, y, BayesianRidge(), energy_within_threshold, 5)

In [ ]:
en.numpy()[en.numpy() > 0.02]

In [ ]:
dataset_path = Path(config.test_datapath)

In [ ]:
struct = {item.name.strip('.json'): read_pymatgen_dict(item) for item in (dataset_path/'structures').iterdir()}

In [ ]:
private_test = pd.DataFrame(columns=['id', 'structures'], index=struct.keys())
private_test = private_test.assign(structures=struct.values())
predictions =[]
for path in tqdm.tqdm(models_path):
    model = prepare_model(
        float(config.cutoff), float(config.lr)
    )
    model.load_weights(path)
    predictions.append(model.predict_structures(private_test.structures))

In [ ]:
test = pd.DataFrame()
test['id'] = private_test['id'].index.values
test['Pred0'] = pd.DataFrame(predictions[0])
test['Pred1'] = pd.DataFrame(predictions[1])
test['Pred2'] = pd.DataFrame(predictions[2])
test['Pred3'] = pd.DataFrame(predictions[3])
test['Pred4'] = pd.DataFrame(predictions[4])
test['Pred5'] = pd.DataFrame(predictions[5])
test['Pred6'] = pd.DataFrame(predictions[6])
test['Pred7'] = pd.DataFrame(predictions[7])
test['Pred8'] = pd.DataFrame(predictions[8])
test['Pred9'] = pd.DataFrame(predictions[9])

In [ ]:
test

In [ ]:
# lasso = LassoLarsCV()
# ridge = BayesianRidge()
lr = LinearRegression()
# gb = GradientBoostingRegressor()

In [ ]:
# lasso.fit(X, y)
# ridge.fit(X, y)
lr.fit(X, y)
# gb.fit(X, y)

In [ ]:
test.drop('id',axis=1,inplace=True)

In [ ]:
# pred1 = lasso.predict(test)
# pred2 = ridge.predict(test)
pred3 = lr.predict(test)
# # pred4 = gb.predict(test)

In [ ]:
prediction_test = pred3

In [ ]:
private_test = private_test.assign(predictions=prediction_test)

In [ ]:
private_test

In [ ]:
sns.distplot(y)

In [ ]:
sns.distplot(private_test['predictions'])

In [ ]:
private_test[['predictions']].to_csv('submission.csv', index_label='id')

In [ ]:
y2 = pd.qcut(y,2, [i for i in range(1,2+1)])

In [ ]:
gml_ml = AutoML()

gml_ml.GMLClassifier(X, y2, metric = accuracy_score, folds = 5)

In [ ]:
def fix_ranges(s):
    c = s['class']
    num = s['y_hat']
    if c == 1:
        return np.clip(num, 0.0929, 0.403)
    elif c == 2:
        return np.clip(num, 0.403, 1.141)
    else:
        return np.clip(num, 1.141, 1.811)        

In [ ]:
def cross_val(X, y, model, rf, metric, folds):
    scores = []
    i = 0
    for tr_in, val_in in KFold(n_splits = folds).split(X, y):
        X_train, y_train, X_val, y_val = X.iloc[tr_in,:], y[tr_in], X.iloc[val_in,:], y[val_in]
        
        y2 = pd.qcut(y_train, 3, [i for i in range(1,3+1)])
        rf.fit(X_train, y2)
        
        X_train['class'] = rf.predict(X_train)
        X_val['class'] = rf.predict(X_val)
        
        model.fit(X_train, y_train)
        
        y_hat = model.predict(X_val)
        
        X_val['y_hat'] = y_hat
        X_val['Pred'] = X_val[['class','y_hat']].apply(fix_ranges,axis=1)
        
        sc = metric(y_val,y_hat)
        print(sc)
        scores.append(sc)
#         if i == 3:
#             break
        i += 1
    return np.mean(scores)

In [ ]:
cross_val(X, y, BayesianRidge(), RandomForestClassifier(), energy_within_threshold, 5)

In [ ]:
y2 = pd.qcut(y, 3, [i for i in range(1,3+1)])

In [ ]:
rf = RandomForestClassifier()
rf.fit(X, y2)

In [ ]:
def fix_ranges(s):
    c = s['class']
    num = s['predictions']
    if c == 1:
        return np.clip(num, 0.0929, 0.403)
    elif c == 2:
        return np.clip(num, 0.403, 1.141)
    else:
        return np.clip(num, 1.141, 1.811)        

In [ ]:
private_test['class'] = rf.predict(test)

In [ ]:
private_test['predictions'] = pred3

In [ ]:
# private_test['predictions'] = private_test[['predictions', 'class']].apply(fix_ranges, axis=1)

In [ ]:
sns.distplot(y)

In [ ]:
sns.distplot(private_test['predictions'])

In [ ]:
private_test[['predictions']].to_csv('submission.csv', index_label='id')